In [2]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv
from tqdm import tqdm

/var/folders/fp/jvqr90jn2gsf_6qw7t0xk7br0000gn/T/ipykernel_29046/910991213.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
load_dotenv()
DATABASE_URL = os.environ.get('DATABASE_URL')
assert DATABASE_URL is not None, 'Missing DATABASE_URL environment variable to connect to the database'
# connect to db
conn = psycopg2.connect(DATABASE_URL)

# Open a cursor to perform database operations
cur = conn.cursor()
cur.execute(
    '''CREATE TABLE IF NOT EXISTS genes (id VARCHAR, 
                                        description VARCHAR,
                                        gene_symbol VARCHAR, 
                                        synonyms VARCHAR )''')

In [3]:
genes_info = pd.read_table('Homo_sapiens.gene_info')
genes_info

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410...,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20231123,-
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899...,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20240107,-
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000291190|AllianceGe...,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20231010,-
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171...,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20231123,-
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156...,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20231225,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193454,741158,8923215,trnD,-,-,-,MT,-,tRNA-Asp,tRNA,-,-,-,-,20200909,-
193455,741158,8923216,trnP,-,-,-,MT,-,tRNA-Pro,tRNA,-,-,-,-,20200909,-
193456,741158,8923217,trnA,-,-,-,MT,-,tRNA-Ala,tRNA,-,-,-,-,20200909,-
193457,741158,8923218,COX1,-,-,-,MT,-,cytochrome c oxidase subunit I,protein-coding,-,-,-,cytochrome c oxidase subunit I,20230818,-


In [4]:
for index, row in tqdm(genes_info.iterrows(), total=genes_info.shape[0]):
    cur.execute('''INSERT INTO genes (id, description, gene_symbol, synonyms) 
                    VALUES  (%s, %s, %s, %s);''', (row['GeneID'], row['description'], row['Symbol'], row['Synonyms']))

conn.commit()
cur.close()
conn.close()

100%|██████████| 193459/193459 [00:11<00:00, 17522.17it/s]


In [5]:
cfde_genesets = pd.read_csv("CFDE Genesets.tsv", sep='\t')
cfde_genesets


,DCC,Library,Link,Hypothesis template
0,Glygen,Glygen Glycosylated Proteins,https://cfde-drc.s3.amazonaws.com/GlyGen/XMT/2...,NaN
1,GTEx,GTEx Tissue-Specific Aging Signatures,https://cfde-drc.s3.amazonaws.com/GTEx/XMT/202...,NaN
2,GTEx,GTEx Tissue Gene Expression Profiles,https://cfde-drc.s3.amazonaws.com/GTEx/XMT/202...,NaN
3,IDG,IDG Drug Targets,https://cfde-drc.s3.amazonaws.com/IDG/XMT/2022...,NaN
4,KOMP2,KOMP2 Mouse Phenotypes,https://cfde-drc.s3.amazonaws.com/KOMP2/XMT/20...,NaN
5,LINCS,LINCS L1000 CMAP Chemical Pertubation Consensu...,https://cfde-drc.s3.amazonaws.com/LINCS/XMT/20...,NaN
6,LINCS,LINCS L1000 CMAP CRISPR Knockout Consensus Sig...,https://cfde-drc.s3.amazonaws.com/LINCS/XMT/20...,NaN
7,MoTrPAC,MoTrPAC Rat Endurance Exercise Training,https://cfde-drc.s3.amazonaws.com/MoTrPAC/XMT/...,NaN
8,Metabolomics,Metabolomics Gene-Metabolite Associations,https://cfde-drc.s3.amazonaws.com/MW/XMT/2022-...,NaN


In [34]:
# read in all genes and convert to dataframe
ingested_genes = pd.read_sql("SELECT * from genes", conn)

# for each line, open with file link and populate database
for index, row in cfde_genesets.iterrows():
    import urllib.request
    with urllib.request.urlopen(row['Link']) as f:
        html = f.read().decode('utf-8') 
        for line in html.split('\n'):
            line_content = line.split('\t')
            geneset_name =  line_content[0]
            genes = line_content[1:]
            genes.remove('')
            for gene_symbol in genes: 
                gene_db_row = ingested_genes.loc[ingested_genes['gene_symbol'] == gene_symbol]
                if not gene_db_row.empty():
                    # do the relation 
                    print(gene_db_row['id'].values[0])
            # print(geneset_name)
            # print(genes)
        break


/var/folders/fp/jvqr90jn2gsf_6qw7t0xk7br0000gn/T/ipykernel_29046/916955059.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ingested_genes = pd.read_sql("SELECT * from genes", conn)


       id gene_symbol       synonyms     description
919  1113        CHGA  CGA|PHE5|PHES  chromogranin A
1113
           id gene_symbol              synonyms  \
27591  727897       MUC5B  MG1|MUC-5B|MUC5|MUC9   

                                  description  
27591  mucin 5B, oligomeric mucus/gel-forming  
727897
        id gene_symbol   synonyms            description
5978  7450         VWF  F8VWF|VWD  von Willebrand factor
7450
        id gene_symbol            synonyms  \
3599  4586      MUC5AC  MUC5|TBM|leB|mucin   

                                  description  
3599  mucin 5AC, oligomeric mucus/gel-forming  
4586
         id gene_symbol                synonyms     description
7989  10216        PRG4  CACP|HAPO|JCAP|MSF|SZP  proteoglycan 4
10216
        id gene_symbol                 synonyms                description
3466  4318        MMP9  CLG4B|GELB|MANDP2|MMP-9  matrix metallopeptidase 9
4318
        id gene_symbol              synonyms    description
2876  3558         IL

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# write relation :                       
# gene__gene_set.writerow(dict(
#                         A=gene_id,
#                         B=gene_set_id,
#                       ))